In [1]:
import numpy as np
import cv2
import os
import math
from random import randint
import utils
from IPython.display import clear_output

augment = True
#Display options
rescale_factor = 2

# Grid parameters
image_w = 256
image_h = 192
grid_w = 8
grid_h = 6
cell_size_x = image_w // grid_w 
cell_size_y = image_h // grid_h

path = "./Data/Faces2"
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]
images = np.zeros((0, image_h, image_w, 3), dtype=np.uint8)
labels = np.zeros((0, grid_h, grid_w, 5), dtype=np.float32)

def Augment(image, r, zoom_min=1.25, zoom_max=2.25, zoom_step=0.25, N_aug = [15, 10, 5, 5, 5], N_aug_hsv=15, N_limit = 50, hue_range=8, sat_range=40, value_range=50):
    aug_images = np.zeros((0, image_h, image_w, 3), dtype=np.uint8)
    aug_labels = np.zeros((0, grid_h, grid_w, 5), dtype=np.float32)
    N_scales = int((zoom_max-zoom_min) // zoom_step + 1)
    h = image.shape[0]
    w = image.shape[1]
    for i in range(N_scales):
        zoom = zoom_min + zoom_step*i
        h_zoom = int(h / zoom)
        w_zoom = int(w / zoom)
        N = 0
        N2 = 0
        while (N < N_aug[i]) and (N2 < N_limit):
            #Generate the random coordinate of top left bounding box vertex
            x1, y1 = randint(0, (w - w_zoom-1)), randint(0, (h - h_zoom-1))
            x2, y2 = (x1+w_zoom), (y1+h_zoom)
            #Select another window if the selected does not contain the bounding box
            if (r[0] <= x1) or (r[1] <= y1) or ((r[0]+r[2]) >= x2) or ((r[1]+r[3]) >= y2):
                N2 += 1
                continue
            sample = cv2.resize(image[y1:y2, x1:x2, :], (w, h))
            r_transformed = r.copy()
            r_transformed[0] = int((r_transformed[0] - x1) * zoom)
            r_transformed[1] = int((r_transformed[1] - y1) * zoom)
            r_transformed[2] = int(r_transformed[2] * zoom)
            r_transformed[3] = int(r_transformed[3] * zoom)
            #Transforming the bounding box coordinates (row_center, col_center, height, width)
            bb = []
            bb.append(r_transformed[1]+r_transformed[3]//2)
            bb.append(r_transformed[0]+r_transformed[2]//2)
            bb.append(r_transformed[3])
            bb.append(r_transformed[2])
            #Get the grid cell coordinate
            cell_x = bb[1] // cell_size_x
            cell_y = bb[0] // cell_size_y
            #Normalize bounding box
            normalized_bb = []
            normalized_bb.append((bb[0]-cell_y*cell_size_y)/cell_size_y)
            normalized_bb.append((bb[1]-cell_x*cell_size_x)/cell_size_x)
            normalized_bb.append(bb[2]/h)
            normalized_bb.append(bb[3]/w)
            label = np.zeros((1, grid_h, grid_w, 5))
            label[0, cell_y, cell_x, 0] = normalized_bb[0] #row_center
            label[0, cell_y, cell_x, 1] = normalized_bb[1] #col_center
            label[0, cell_y, cell_x, 2] = normalized_bb[2] #height
            label[0, cell_y, cell_x, 3] = normalized_bb[3] #width
            label[0, cell_y, cell_x, 4] = 1 #confidence
            #Augmenting with random hsv shifts
            for _ in range(N_aug_hsv):
                delta_h = randint(-hue_range, hue_range)
                delta_s = randint(-sat_range, sat_range)
                delta_v = randint(-value_range, value_range)
                sample = utils.shift_hsv(sample, delta_h, delta_s, delta_v)
                aug_images = np.append(aug_images, sample.reshape((1, h, w, 3)), axis=0)
                aug_labels = np.append(aug_labels, label, axis=0)
            N += 1
            N2 += 1
    return aug_images, aug_labels

i = 1

for file in files:
    print("Labeling image %d/%d"%(i, len(files)))
    img = cv2.resize(cv2.imread(file), (rescale_factor*image_w, rescale_factor*image_h))
    #Select the bounding box
    r = cv2.selectROI(img)
    img = cv2.resize(img, (image_w, image_h))
    if augment:
        r = list(r)
        r[0] = r[0] // rescale_factor
        r[1] = r[1] // rescale_factor
        r[2] = r[2] // rescale_factor
        r[3] = r[3] // rescale_factor
        aug_images, aug_labels = Augment(img, r, N_aug=[10, 5, 5, 5, 5], N_aug_hsv=8)
        images= np.append(images, aug_images, axis=0)
        labels = np.append(labels, aug_labels, axis=0)
    else:
        #Transforming the bounding box coordinates (row_center, col_center, height, width)
        bb = []
        bb.append((r[1]+r[3]//2)//rescale_factor)
        bb.append((r[0]+r[2]//2)//rescale_factor)
        bb.append(r[3]//rescale_factor)
        bb.append(r[2]//rescale_factor)
        #Get the grid cell coordinate
        cell_x = bb[1] // cell_size_x
        cell_y = bb[0] // cell_size_y
        #Normalize bounding box
        normalized_bb = []
        normalized_bb.append((bb[0]-cell_y*cell_size_y)/cell_size_y)
        normalized_bb.append((bb[1]-cell_x*cell_size_x)/cell_size_x)
        normalized_bb.append(bb[2]/image_h)
        normalized_bb.append(bb[3]/image_w)
        label = np.zeros((1, grid_h, grid_w, 5))
        label[0, cell_y, cell_x, 0] = normalized_bb[0] #row_center
        label[0, cell_y, cell_x, 1] = normalized_bb[1] #col_center
        label[0, cell_y, cell_x, 2] = normalized_bb[2] #height
        label[0, cell_y, cell_x, 3] = normalized_bb[3] #width
        label[0, cell_y, cell_x, 4] = 1 #confidence
        images= np.append(images, img.reshape(1, image_h, image_w, 3), axis=0)
        labels = np.append(labels, label, axis=0)
    cv2.destroyAllWindows()
    clear_output(wait=True)
    print("Images in dataset: ", images.shape[0])
    i += 1

print("Saving %d images"%images.shape[0])
np.save("Data/Processed/myface3_images", images)
np.save("Data/Processed/myface3_labels", labels)

Images in dataset:  21520
Labeling image 98/98


error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\resize.cpp:3289: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


In [2]:
np.save("Data/Processed/myface3_images", images)
np.save("Data/Processed/myface3_labels", labels)